In [2]:
#토크나이저 블록

from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english" #토크나이저 로드 -> AutoTokenizer 활용
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs) #토크나이저는 토큰화, 입력id 변환을 동시에 일괄 수행함

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [4]:
#그러나 모델 입력으로 넣기 위해서는 단일 시퀀스 입력은 에러
#여러 연속의 시퀀스 입력이 필요함

sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)
print(model_inputs)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [5]:
# 입력된 모든 시퀀스 중 가장 긴 시퀀스의 길이에 맞춰 짧은 시퀀스들의 뒷부분을 패딩
#tokenizer의 padding 매개변수에 "longest" 값 전달, True 값을 전달해도 동일한 작업 수행
#padding 매개변수에 truncation 전달 시 반대로 긴 시퀀스의 길이를 잘라내는 작업 수행
model_inputs = tokenizer(sequences, padding="longest")

# 모델 디폴트로 설정된 maxlength로 텍스트 청킹
# BERT or DistilBERT 의 경우 디폴트 길이가 512
model_inputs = tokenizer(sequences, padding="max_length")

# 사용자가 지정하는 maxlength를 매개변수로 전달하면 원하는 길이로 청킹 가능
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)


# 모델 디폴트로 설정된 maxlength 이상의 길이의 문장을 잘라내는 청킹
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# 사용자가 지정하는 maxlength를 매개변수로 전달하면 원하는 길이로 청킹 가능
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

In [7]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")
print(model_inputs)

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")
print(model_inputs)

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")
print(model_inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}
{'input_ids': array([[  101,  1045,  1005,  2310,  2042,

In [8]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"]) # 문장에 포함된 시작 토큰과 마무리 토큰 eos 가 포함

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens) #문장 자체의 id만 보여줌
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [9]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]